# Diversity of defence mechanism in cow rumen microbiome

In [1]:
#bash funtion and main path
def bash(string,name,queue):
  script = "%s.sh"%name
  with open(script, "w") as text_file:
    text_file.write(string)
  !chmod +x $script
  !qsub -q $queue $script
  #!rm $script
  
ws = '/beegfs/work/workspace/ws/ho_kezau83-rumen_df-0/'


In [4]:
#control the submitted jobs
!qstat -u ho_kezau83


mgmt02: 
                                                                                  Req'd       Req'd       Elap
Job ID                  Username    Queue    Jobname          SessID  NDS   TSK   Memory      Time    S   Time
----------------------- ----------- -------- ---------------- ------ ----- ------ --------- --------- - ---------
11076150                ho_kezau83  smp      gtdbtk_classify.    --      1     20     150gb  72:00:00 Q       -- 
11076161                ho_kezau83  long     gtdbtk_classify.   1514     1     16      64gb  96:00:00 R  00:46:18


In [ ]:
#delete specific jobs
!qdel 10904183

In [ ]:
#delete all jobs in queue
!qselect -u ho_kezau83 -s Q | xargs qdel

In [ ]:
#delte files
!rm *cctyper.sh*

## Download and clean the Hungate1000 genomes

In [ ]:
#Create list of file indexes to access it later
import os
!rm -r bulk_data_91600/.ipynb_checkpoints
genomes = sorted(list(f.split('.')[0] for f in os.listdir('bulk_data_91600-2')))

In [ ]:
for genome in genomes:
    file_in = 'bulk_data_91600-2/%s/Assembly/QC_and_Genome_Assembly/finalContigs.fasta' %genome
    file_out = 'genomes_clean/%s.fasta' %genome
    !mv $file_in $file_out

## Download and clean the Mgnify genomes

In [ ]:
#Dowload the genomes using a bash line
for genome in $(tail -n +2 genomes-all_metadata.csv | cut -d$'\t' -f 20 ); do wget $genome; done


In [ ]:
#Create list of file indexes to access it later
import os
!rm -r genomes/.ipynb_checkpoints
genomes = sorted(list(f.split('.')[0] for f in os.listdir('hq_genomes/')))

In [ ]:
#Extract only the contigs from the GFF file
out_dir = ws+'/fasta_genomes'
for genome in genomes:
    file = ws+'/mags/%s.gff' %genome
    out_file = out_dir+'/%s.fasta' %genome
    !sed -n '/##FASTA/ ,$p' $file | sed '1d' > $out_file

## Download Phage db https://github.com/RyanCook94/inphared

In [ ]:
!wget http://inphared.s3.climb.ac.uk/1Apr2022_genomes_excluding_refseq.fa

## Download phage JGI DB

In [ ]:
#selecet only reference genomes using R
library(tidyverse)

sed <- read_tsv("IMG_VR-3/IMG_VR_2020-10-12_5.1/IMGVR_all_Sequence_information.tsv")
 colnames(sed)[1] <- "Genome

filter_vir <- sed %>% 
  filter(`MIUViG quality`=="Reference") %>% 
  select(Genome)

write_delim(filter_vir, file = "list_virus_jgi.txt", delim = "\t")

In [ ]:
#linearize db the db
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=08:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash

awk '/^>/ {printf("%s%s\t",(N>0?"\n":""),$0);N++;next;} {printf("%s",$0);} END {printf("\n");}' < %s > %s
'''

path_db = ws+'/IMGVR_all_nucleotides.fna'
out_file = ws+'/linear_IMGVR_reference.fna'
bash(string%(path_db, out_file),'linear_fasta','short')

In [ ]:
#filter the db using the id_list created using R. Only References genomes will be kept
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=08:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash

cat %s | grep -A 1 -f %s > %s
'''

path_db = ws+'/linear_IMGVR_all_nucleotides.fna'
list_file = ws+'/list_virus_jgi_2.txt'
out_file = ws+'/IMGVR_reference.fna'
bash(string%(path_db,list_file, out_file),'grep','short')

## Genomes QC

In [ ]:
#check genomes QC, split genomes in 1000
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=48:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate checkm2
checkm2 predict --threads 14 \
                --input %s \
                --extension .fasta \
                --output-directory %s \
                --database_path /beegfs/work/workspace/ws/ho_kezau83-conda-0/checkm2/CheckM2_database/uniref100.KO.1.dmnd \
                --tmpdir %s
conda deactivate
'''
genomes_path = ws+'/archive_genomes/formatted_genomes/'
out_file = ws+'/checkm2'
temdir = ws+'/temp'
bash(string%(genomes_path, out_file, temdir),'checkM2_low','short')

## Taxonomic annotation of genomes

In [ ]:
string ='''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=48:00:00
#PBS -l mem=150gb
#PBS -q smp
#PBS -S /bin/bash
# Activate conda enviroment
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate gtdbtk
# Run
gtdbtk classify_wf --genome_dir %s --extension fasta --out_dir %s --cpus 20
'''
path_genomes = ws+'/archive_genomes/formatted_genomes/'
out_dir = ws+'/gtdb_classify_all'
!mkdir $out_dir
bash(string%(path_genomes, out_dir), 'gtdbtk_classify', 'smp')

In [ ]:
hq_genomes = open('hq_genomes.txt').read().splitlines()
hq_genomes

In [ ]:
for hq_genome in hq_genomes:
    file_in = 'archive_genomes/formatted_genomes/%s.fasta' %hq_genome
    file_out = 'hq_genomes/%s.fasta' %hq_genome
    !cp $file_in $file_out

## Phylogeny and tree construction

In [ ]:
#config file
string = '''
#PBS -l nodes=1:ppn=1
#PBS -l walltime=01:00:00
#PBS -l mem=10gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate phylophlan

phylophlan_write_config_file -o %s \
    -d a \
    --db_aa diamond \
    --map_aa diamond \
    --map_dna diamond \
    --msa mafft \
    --tree1 fasttree \
    --tree2 raxml

conda deactivate
'''
out_file = ws+'/custom_config_nt.cfg'
bash(string%(out_file),'phylophlan_custom','short')


In [ ]:
#run Phylophlan
string = '''
#PBS -l nodes=1:ppn=30
#PBS -l walltime=72:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate phylophlan

cd /beegfs/work/workspace/ws/ho_kezau83-rumen_df-0/phylophlan


python /beegfs/work/workspace/ws/ho_kezau83-conda-0/phylophlan/phylophlan/phylophlan.py -i %s \
    --output_folder %s \
    --genome_extension .fasta \
    -d phylophlan \
    -t a \
    --diversity high \
    --accurate \
    -f %s \
    --nproc 30 \
'''
!mkdir phylophlan
genomes = ws+'/padloc/faa'
out_file = ws+'/phylophlan'
config = ws+'/custom_config_nt.cfg'
bash(string%(genomes, out_file, config),'phylophlan','smp')

In [ ]:
!rm -r phylophlan/

In [ ]:
import os
hq_genomes = sorted(list(f.split('.')[0] for f in os.listdir('hq_genomes//')))
hq_genomes

## Annotation of defence systems

In [ ]:
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=08:00:00
#PBS -l mem=10gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate padloc
cd /beegfs/work/workspace/ws/ho_kezau83-rumen_df-0/padloc_chicken
padloc --fna %s --cpu 10
conda deactivate
'''
!mkdir padloc
for hq_genome in hq_genomes:
    path_genomes_fasta = ws+'/chickens_all/%s.fasta' %hq_genome
    bash(string%(path_genomes_fasta),'%s_padloc'%hq_genome,'short')

In [ ]:
!mv padloc_isolates/*.csv padloc_csv/

In [ ]:
!zip chicken_virus padloc_chicken/*csv

In [ ]:
!ls padloc_csv | wc

In [ ]:
!rm *padloc.sh*

In [ ]:
!mv *padloc.sh* padloc_sh/

## Prediction of lysogenic Phages

In [ ]:
# add genome names in the headers of contigs for the hungate collection

for genome in genomes:
    file = ws+'/fasta_genomes/%s.fasta' % genome
    out = ws+'/formatted_genomes/%s.fasta' % genome
    header = r"s/>/>'{}'_/g".format(genome) 
    !sed $'{header}' $file > $out 

In [ ]:
!cat hq_genomes/* > all_genomes.fasta

In [ ]:
#virsoter2
#find the lysogenic phages in the genomes
string = '''
#PBS -l nodes=1:ppn=30
#PBS -l walltime=8:00:00:00
#PBS -l mem=300gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate vs2

#run program
virsorter run --keep-original-seq -i %s -w %s --min-length 9999 -j 30 all
'''
in_file = ws+'/all_genomes.fasta'
out_file = ws+'/vs2_out_all'
!mkdir $out_file
bash(string%(in_file, out_file), 'vs2', 'smp')

In [ ]:
#checkV
#clean the viral contigs
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=72:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate checkv

checkv end_to_end %s %s -t 20 -d %s

conda deactivate
'''
in_file = ws+'/vs2_out_all/final-viral-combined.fa'
db = '/beegfs/work/workspace/ws/ho_kezau83-conda-0/checkv_db/checkv-db-v1.0'
out_file = ws+'/checkv'
!mkdir $out_file
bash(string%(in_file, out_file, db), 'checkv', 'smp')

In [ ]:
#prodigal
#redict genes and proteins
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=2:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate padloc

prodigal -i %s -o %s -a %s  -p meta

conda deactivate
'''
in_file = ws+'/checkv/filtered_proviruses.fna'
outfile1 = ws+'/checkv/filtered_proviruses.gbk'
outfile2 = ws+'/checkv/filtered_proviruses.faa'
bash(string%(in_file, outfile1, outfile2), 'prodigal', 'short')

# Annotation of Proviruses

In [ ]:
#vcontact2
#redict genes and proteins
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=2:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate vcontact2

vcontact2_gene2genome -p %s -o %s -s 'Prodigal-FAA'

conda deactivate
'''

infile = ws+'/checkv/clean_filtered_proviruses.faa'
outfile = ws+'/checkv/viral_genomes_g2g.csv'
bash(string%(infile, outfile), 'vcontact2', 'short')

In [ ]:
#vcontact2
#redict genes and proteins
string = '''
#PBS -l nodes=1:ppn=25
#PBS -l walltime=48:00:00
#PBS -l mem=250gb
#PBS -S /bin/bash

module load devel/java_jdk/11.0.4
#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate vContact2

vcontact2 --raw-proteins %s \
--rel-mode Diamond \
--proteins-fp %s \
--db 'ProkaryoticViralRefSeq97-Merged' \
--pcs-mode MCL --vcs-mode ClusterONE \
--c1-bin /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/envs/vcontact2/bin/cluster_one-1.0.jar \
--output-dir %s \
-t 25

conda deactivate
'''
infile = ws+'/checkv/clean_filtered_proviruses.faa'
g2g = ws+'/checkv/viral_genomes_g2g.csv'
outfile = ws+'/vcontact2'
!mkdir $outfile
bash(string%(infile, g2g, outfile), 'vcontact2', 'smp')

## CRISPRcasTyper 

In [ ]:
#done
genomes_500 = genomes[1:500]
genoms_1500
genomes_2500
genomes_3500
genomes_5500

In [ ]:
genomes_1 = genomes[2:3]
genomes_1

In [ ]:
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=2:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate cctyper

cctyper %s %s --prodigal meta --no_plot -t 10

conda deactivate
'''
!mkdir cctyper
for hq_genome in hq_genomes:
    path_genomes_fasta = ws+'/hq_genomes/%s.fasta' %hq_genome
    out_path = ws+'/cctyper/%s' %hq_genome
    bash(string%(path_genomes_fasta, out_path, ),'%s_cctyper'%hq_genome,'short')

In [ ]:
# move sapcer to the new directory 
!mkdir spacers
for hq_genome in hq_genomes:
  oldass = ws+'/cctyper/%s/spacers/*' %hq_genome
  newass = ws+'/spacers/'
  !cp $oldass $newass

In [ ]:
# create spacers files
!cat spacers/* > all_spacers.fasta

In [ ]:
#count spacers
!cat all_spacers.fasta | grep ">" | wc

In [ ]:
#remove spacers from folder
!rm -r spacers

In [ ]:
# move CRISPR-Cas-loci to the new directory 
!mkdir crispr_cas_loci
for hq_genome in hq_genomes:
  oldass = ws+'cctyper/%s/CRISPR_Cas.tab' %hq_genome
  newass = ws+'crispr_cas_loci/%s_CRISPR_Cas.tab' %hq_genome
  !cp $oldass $newass

In [ ]:
!zip all_cas_loci.zip crispr_cas_loci/*

In [ ]:
# move CRISPR-orphan to the new directory 
!mkdir crispr_orphan
for genome in genomes:
  oldass = ws+'/cctyper/%s/crisprs_orphan.tab' %genome
  newass = ws+'/crispr_orphan/%s_crisprs_orphan.tab' %genome
  !cp $oldass $newass

In [ ]:
# move cas operons to the new directory 
!mkdir cas_orphan
for genome in genomes:
  oldass = ws+'/cctyper/%s/cas_operons_orphan.tab' %genome
  newass = ws+'/cas_orphan/%s_cas_operons_orphan.tab' %genome
  !cp $oldass $newass

## Match spacers againts phage db

In [ ]:
#download rumen phage db
!wget https://zenodo.org/record/7412085/files/RVD.fa?download=1

In [ ]:
#rename db
!mv RVD.fa\?download\=1 rumen_phages.fasta

In [ ]:
# make a database file
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=24:00:00
#PBS -l mem=80gb
#PBS -S /bin/bash
module load bio/blastplus/2.5.0
makeblastdb -in %s -out %s -parse_seqids -dbtype nucl
'''
file_path = ws+'rumen_phages.fasta'
db_path = ws+'rumen_phages/phages'
bash(string%(file_path, db_path),'makedb', 'short')


In [ ]:
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=24:00:00
#PBS -l mem=80gb
#PBS -S /bin/bash
module load bio/blastplus/2.5.0
blastn -task blastn-short -query %s -db %s -out %s -perc_identity 95 -max_target_seqs 1 -num_threads 16 -evalue 1e-4 -outfmt "6 qseqid sseqid pident qcovs length mismatch gapopen qstart qend sstart send evalue bitscore"

'''
file_path = ws+'all_spacers.fasta'
db_path = ws+'rumen_phages/phages'
out_file = ws+'spacers_match_proviruses.txt'
bash(string%(file_path, db_path,out_file),'blastn', 'short')

In [ ]:
!rm -r phylophlan/*

In [ ]:
!mkdir genomes_prodigal

In [ ]:
!mv padloc_new_db/*.faa genomes_prodigal/

In [ ]:
genomes_1=genomes[1:2]
genomes_1

In [ ]:
#ccfinder
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=2:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash

export SINGULARITY_CACHEDIR=/beegfs/work/workspace/ws/ho_kezau83-cow_rumen-0/ccfinder
cd /beegfs/work/workspace/ws/ho_kezau83-cow_rumen-0/ccfinder
singularity exec -B $PWD /beegfs/work/workspace/ws/ho_kezau83-cow_rumen-0/CrisprCasFinder.simg perl /usr/local/CRISPRCasFinder/CRISPRCasFinder.pl -so /usr/local/CRISPRCasFinder/sel392v2.so -cf /usr/local/CRISPRCasFinder/CasFinder-2.0.3 -drpt /usr/local/CRISPRCasFinder/supplementary_files/repeatDirection.tsv -rpts /usr/local/CRISPRCasFinder/supplementary_files/Repeat_List.csv -cas -def G -out %s -in %s
'''
!mkdir ccfinder
for genome in genomes_1:
    path_genomes_fasta = ws+'/formatted_genomes/%s.fasta' %genome
    out_path = ws+'/ccfinder/%s' %genome
    bash(string%(out_path, path_genomes_fasta),'%s_ccfinder'%genome,'short')


In [ ]:
genomes6 = genomes[5000:6000]

In [ ]:
for genome in genomes6:
  oldass = ws+'/formatted_genomes/%s.fasta' %genome
  newass = ws+'/new_analysis/genomes/genomes6/%s.fasta' %genome
  !sed -e 's/\r$//' $oldass > $newass

In [ ]:
genomes_90complet = open('new_analysis/genomes/90complet_list.txt').read().splitlines()


In [ ]:
for genome in genomes_90complet:
  oldass = ws+'/new_analysis/genomes/all_genomes/%s.fasta' %genome
  newass = ws+'/new_analysis/genomes/90complet_genomes/'
  !mv $oldass $newass

In [ ]:
!mkdir archive_cctyper

In [ ]:
!wget https://ccb-microbe.cs.uni-saarland.de/plsdb/plasmids/download/plsdb.fna.bz2

In [ ]:
!bzip2 -d plsdb.fna.bz2

In [ ]:
!cat checkv/filtered_viral_listid.txt | sed 's/^/>/g' > checkv/filtered_id.txt

In [ ]:
!cat checkv/proviruses.fna | sed 's/ .*//g' > checkv/clean_proviruses.fna

In [ ]:
!cat checkv/clean_proviruses.fna | grep -A 1 -f checkv/filtered_id.txt > checkv/filtered_proviruses.fna

In [ ]:
!cat checkv/filtered_proviruses.fna | grep ">" | wc

In [ ]:
!cat checkv/filtered_proviruses.faa | sed 's/||/_/g' > checkv/clean_filtered_proviruses.faa

# Individual tress

In [ ]:
prevotella_genomes = open('list_genomes_trees/Prevotella_list.txt').read().splitlines()
prevotella_genomes

In [ ]:
for prevotella_genome in prevotella_genomes:
    file_in = 'hq_genomes/%s.fasta' %prevotella_genome
    file_out = 'prevotella//%s.fasta' %prevotella_genome
    !cp $file_in $file_out

In [ ]:
limimorpha_genomes = open('list_genomes_trees/Limimorpha_list.txt').read().splitlines()
limimorpha_genomes

In [ ]:
for limimorpha_genome in limimorpha_genomes:
    file_in = 'hq_genomes/%s.fasta' %limimorpha_genome
    file_out = 'Limimorpha/%s.fasta' %limimorpha_genome
    !cp $file_in $file_out

In [ ]:
butyvibrio_genomes = open('list_genomes_trees/Butirivibrio_list.txt').read().splitlines()
butyvibrio_genomes

In [ ]:
for butyvibrio_genome in butyvibrio_genomes:
    file_in = 'hq_genomes/%s.fasta' %butyvibrio_genome
    file_out = 'butyvibrio/%s.fasta' %butyvibrio_genome
    !cp $file_in $file_out

In [ ]:
rumino_genomes = open('list_genomes_trees/Ruminococcus_list.txt').read().splitlines()
rumino_genomes

In [ ]:
for rumino_genome in rumino_genomes:
    file_in = 'hq_genomes/%s.fasta' %rumino_genome
    file_out = 'ruminococcus/%s.fasta' %rumino_genome
    !cp $file_in $file_out

In [ ]:
string = '''
#PBS -l nodes=1:ppn=12
#PBS -l walltime=4:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate drep
dRep dereplicate -d -p 12 %s -g %s -comp 89 -con 10 --genomeInfo %s -sa 0.95
'''
!mkdir ruminococcus/drep
drep_out = ws+'/Limimorpha/drep95'
genomes_path = ws+'/Limimorpha/*.fasta'
checkm = ws+'/list_genomes_trees/checkm_limimorpha.txt'
bash(string%(drep_out, genomes_path, checkm), 'drep', 'short')

In [ ]:
!zip all_prevotella prevotella/drep/dereplicated_genomes/*

# Metabolic annotation

In [ ]:
!unzip dram_chickens/assemblies.zip -d dram_chickens

In [ ]:
# annotation with DRAM
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate DRAM

#run program
DRAM.py annotate -i '/beegfs/work/workspace/ws/ho_kezau83-rumen_df-0/dram_chickens/assemblies/*.fasta' -o %s
'''

out_folder= ws+'/dram_chickens/dram_out'
bash(string%(out_folder), 'dram', 'smp')

In [ ]:
#distill functions with DRAM
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=06:00:00
#PBS -l mem=40gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate DRAM

#run program
DRAM.py distill -i %s -o %s --trna_path %s --rrna_path %s 
'''
in_file = ws+'/dram_chickens/dram_out/annotations.tsv'
out_folder= ws+'/dram_chickens/dram_out/distilled'
trna =  ws+'/dram_chickens/dram_out/trnas.tsv'
rrna = ws+'/dram_chickens/dram_out/rrnas.tsv'
bash(string%(in_file, out_folder, trna,rrna), 'dram-distilled', 'short')

In [ ]:
!zip all_limimorpha.zip Limimorpha/drep95/dereplicated_genomes/*

In [ ]:
#mags chicken

In [3]:
string ='''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=96:00:00
#PBS -l mem=64gb
#PBS -q long
#PBS -S /bin/bash
# Activate conda enviroment
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate gtdbtk-2.3
# Run
gtdbtk classify_wf --genome_dir %s --extension fasta --out_dir %s --cpus 20 --skip_ani_screen
'''
path_genomes = ws+'archive_genomes/formatted_genomes/'
out_dir = ws+'gtdb_new_defence_long'
!mkdir $out_dir
bash(string%(path_genomes, out_dir), 'gtdbtk_classify', 'long')

11076161


In [ ]:
!rename '.fna' '' chicken_mags/*

In [ ]:
for genome in $(tail -n +2  | cut -d$'\t' -f 20 ); do wget $genome; done